In [1]:
import os

os.listdir ("./data")

['First and Future Data Manual 12062018.docx',
 'game_data.csv',
 'NGS-2016-post.csv',
 'NGS-2016-pre.csv',
 'NGS-2016-reg-wk1-6.csv',
 'NGS-2016-reg-wk13-17.csv',
 'NGS-2016-reg-wk7-12.csv',
 'NGS-2017-post.csv',
 'NGS-2017-pre.csv',
 'NGS-2017-reg-wk1-6.csv',
 'NGS-2017-reg-wk13-17.csv',
 'NGS-2017-reg-wk7-12.csv',
 'player_punt_data.csv',
 'play_information.csv',
 'play_player_role_data.csv',
 'video_footage-control.csv',
 'video_footage-injury.csv',
 'video_review.csv',
 '~$rst and Future Data Manual 12062018.docx']

In [54]:
import pandas as pd

df = pd.read_csv ("./data/NGS-2017-reg-wk1-6.csv", parse_dates = ['Time'])

df.sort_values (['GameKey', 'PlayID', 'GSISID', 'Time'], inplace = True)

df.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event
5144770,2017,399,1336,23449.0,2017-09-08 01:31:11.000,41.560001,54.279999,0.13,78.879997,192.050003,NaN
5144771,2017,399,1336,23449.0,2017-09-08 01:31:11.100,41.540001,54.139999,0.14,77.760002,190.759995,NaN
5144772,2017,399,1336,23449.0,2017-09-08 01:31:11.200,41.509998,54.000000,0.14,76.459999,189.559998,NaN
5144773,2017,399,1336,23449.0,2017-09-08 01:31:11.300,41.490002,53.860001,0.14,75.800003,188.449997,NaN
5144320,2017,399,1336,23449.0,2017-09-08 01:31:11.400,41.470001,53.709999,0.15,75.769997,187.589996,NaN


In [70]:
groupe = df.groupby (['Season_Year','GameKey', 'PlayID', 'GSISID'])["Time","Event"].count()
groupe.columns = ['Time_Player', 'Nb_event_played']
groupe.head(5)

Time_Player  Nb_event_played
Season_Year GameKey PlayID GSISID                               
2017        399     1336   23449.0           29                1
                           23534.0          205                6
                           26035.0          205                6
                           27010.0          205                6
                           27442.0          205                6

In [71]:
groupe.reset_index(inplace=True)
groupe.head(5)



,Season_Year,GameKey,PlayID,GSISID,Time_Player,Nb_event_played
0,2017,399,1336,23449.0,29,1
1,2017,399,1336,23534.0,205,6
2,2017,399,1336,26035.0,205,6
3,2017,399,1336,27010.0,205,6
4,2017,399,1336,27442.0,205,6


In [73]:
groupy = groupe.groupby(["Season_Year","GameKey","PlayID"])['Time_Player',"Nb_event_played"].max()
groupy.columns = ["Duration","Max_number_event"]
groupy.reset_index(inplace=True)
groupy.head(5)

,Season_Year,GameKey,PlayID,Duration,Max_number_event
0,2017,399,1336,205,6
1,2017,399,1776,862,10
2,2017,399,1884,971,8
3,2017,399,2572,212,6
4,2017,399,2712,663,11


In [74]:
select_player = pd.merge(groupe, groupy, on = ["Season_Year","GameKey","PlayID"], how = "inner")
select_player.head(4)#The shape are checked to be sure of changes

,Season_Year,GameKey,PlayID,GSISID,Time_Player,Nb_event_played,Duration,Max_number_event
0,2017,399,1336,23449.0,29,1,205,6
1,2017,399,1336,23534.0,205,6,205,6
2,2017,399,1336,26035.0,205,6,205,6
3,2017,399,1336,27010.0,205,6,205,6


In [83]:
select_player = select_player.loc[(select_player.Time_Player == select_player.Duration )]
select_player.shape[0]

19758

In [7]:
first = groupe.loc[(groupe.GameKey == 399)&(groupe.PlayID == 1336)&(groupe.Time == 205)].count()
first

GameKey        22
PlayID         22
GSISID         22
Season_Year    22
Time           22
x              22
y              22
dis            22
o              22
dir            22
Event          22
dtype: int64

In [15]:
set(groupe.loc[(groupe.GameKey == 399)]['PlayID'])

{1336,
 1776,
 1884,
 2572,
 2712,
 2943,
 3029,
 3312,
 3348,
 3384,
 4006,
 4133,
 4429,
 4604}

In [17]:
second = groupe.loc[(groupe.GameKey == 399)&(groupe.PlayID == 1776)&(groupe.Time == 862)].count()
second

GameKey        22
PlayID         22
GSISID         22
Season_Year    22
Time           22
x              22
y              22
dis            22
o              22
dir            22
Event          22
dtype: int64

In [19]:
set(df['Event'])

{nan,
 'pass_outcome_interception',
 'man_in_motion',
 'line_set',
 'run',
 'fumble_offense_recovered',
 'punt_play',
 'punt_muffed',
 'punt_land',
 'fumble_defense_recovered',
 'fumble',
 'timeout',
 'shift',
 'penalty_flag',
 'pass_forward',
 'kick_received',
 'out_of_bounds',
 'punt',
 'pass_outcome_caught',
 'huddle_start_offense',
 'snap_direct',
 'punt_blocked',
 'penalty_declined',
 'timeout_tv',
 'lateral',
 'punt_downed',
 'ball_snap',
 'touchback',
 'punt_received',
 'penalty_accepted',
 'touchdown',
 'tackle',
 'pass_arrived',
 'pass_outcome_incomplete',
 'play_submit',
 'punt_fake',
 'fair_catch',
 'huddle_break_offense',
 'first_contact'}

In [20]:
"""Let's analyse the plays and select only the punt plays to begin with"""
punt_play = pd.read_csv("./data/play_information.csv")
punt_play.head(10)

,Season_Year,Season_Type,GameKey,Game_Date,Week,PlayID,Game_Clock,YardLine,Quarter,Play_Type,Poss_Team,Home_Team_Visit_Team,Score_Home_Visiting,PlayDescription
0,2016,Pre,2,08/13/2016,2,191,12:30,LA 47,1,Punt,LA,LA-DAL,0 - 7,"(12:30) J.Hekker punts 52 yards to DAL 1, Cent..."
1,2016,Pre,2,08/13/2016,2,1132,12:08,LA 29,2,Punt,LA,LA-DAL,7 - 21,"(12:08) J.Hekker punts 51 yards to DAL 20, Cen..."
2,2016,Pre,2,08/13/2016,2,1227,10:01,DAL 18,2,Punt,DAL,LA-DAL,7 - 21,"(10:01) C.Jones punts 40 yards to LA 42, Cente..."
3,2016,Pre,2,08/13/2016,2,1864,00:21,DAL 46,2,Punt,LA,LA-DAL,7 - 24,"(:21) J.Hekker punts 31 yards to DAL 15, Cente..."
4,2016,Pre,2,08/13/2016,2,2247,10:26,DAL 15,3,Punt,DAL,LA-DAL,14 - 24,"(10:26) M.Wile punts 40 yards to LA 45, Center..."
5,2016,Pre,2,08/13/2016,2,2342,09:00,LA 50,3,Punt,LA,LA-DAL,14 - 24,"(9:00) J.Hekker punts 50 yards to end zone, Ce..."
6,2016,Pre,2,08/13/2016,2,2805,00:27,LA 38,3,Punt,LA,LA-DAL,14 - 24,"(:27) J.Hekker punts 48 yards to DAL 14, Cente..."
7,2016,Pre,2,08/13/2016,2,2970,13:30,LA 41,4,Punt,DAL,LA-DAL,14 - 24,"(13:30) M.Wile punts 41 yards to end zone, Cen..."
8,2016,Pre,2,08/13/2016,2,3439,05:28,DAL 15,4,Punt,DAL,LA-DAL,21 - 24,"(5:28) M.Wile punts 59 yards to LA 26, Center-..."
9,2016,Pre,3,08/11/2016,2,455,06:44,BLT 32,1,Punt,BLT,BLT-CAR,0 - 3,(6:44) (Punt formation) S.Koch punts 54 yards ...


In [21]:
set(punt_play['Play_Type'])

{'Punt'}

In [23]:
simple_play_info = pd.DataFrame()
simple_play_info['Season_Year']=punt_play['Season_Year']
simple_play_info['GameKey'] = punt_play['GameKey']
simple_play_info['PlayID'] = punt_play['PlayID']
simple_play_info.head(5)


,Season_Year,GameKey,PlayID
0,2016,2,191
1,2016,2,1132
2,2016,2,1227
3,2016,2,1864
4,2016,2,2247


In [45]:
merged_plays = pd.merge(df, simple_play_info, on = ["Season_Year","GameKey","PlayID"], how = "inner")

In [33]:
print(merged_plays.shape)
df.shape

(9422296, 11)


(9433922, 11)

In [66]:
new_merged_plays =  pd.merge(merged_plays, groupy, on = ["Season_Year","GameKey","PlayID"], how = "inner")

In [67]:
new_merged_plays.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Duration,Max_number_event
0,2017,399,1336,23449.0,2017-09-08 01:31:11.000,41.560001,54.279999,0.13,78.879997,192.050003,NaN,205,6
1,2017,399,1336,23449.0,2017-09-08 01:31:11.100,41.540001,54.139999,0.14,77.760002,190.759995,NaN,205,6
2,2017,399,1336,23449.0,2017-09-08 01:31:11.200,41.509998,54.000000,0.14,76.459999,189.559998,NaN,205,6
3,2017,399,1336,23449.0,2017-09-08 01:31:11.300,41.490002,53.860001,0.14,75.800003,188.449997,NaN,205,6
4,2017,399,1336,23449.0,2017-09-08 01:31:11.400,41.470001,53.709999,0.15,75.769997,187.589996,NaN,205,6


In [ ]:
"""Regarder s'il y a tjs un snapp ball à chaque partie"""